<a href="https://colab.research.google.com/github/Rachalisda/etf_analysis/blob/main/Best_Dividends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Portfolio management to calculate the best ETFs to buy
1. Find best portfolio that maximizes profit and mimizes risk
2. Update the system bi-monthly, or find another length of time to check in to maximize profits
3. Build a dashboard to display this - essentially creating an auto etf screener

4. Use and show it off!

In [7]:
import pandas as pd
# Variables
time_frame = '3mo'
initial_investment = 5000

# Define the portfolio DataFrame with ETF tickers and investment amounts
portfolio = pd.DataFrame({
    #'ETF': ['CUBE', 'EEMV', 'HTGC', 'JEPI', 'JEPQ', 'MAIN', 'QQQM', 'SCHD','IBIT','SHLD','RING','IAK','MAGS','SMH','USD','DHT','FBL','VUG','VGT','MGK','SFY','XSD','VTI','SPY','IVV','VOO','VTI','QQQ','VEA','VUG','VTV','IEFA','AGG','BND','IWF','IJH','SBR']
    #'ETF': ['CUBE', 'RING', 'SHLD', 'IAK', 'AGG', 'BND', 'SCHD', 'EEMV', 'VTV', 'JEPI','DIA','VTI','VTWO','QQQ','NASDX','SWPPX','IVV','SPY','VOO','FNILX']
    'ETF' : [
    "CUBE", "RING", "SHLD", "IAK", "AGG", "BND", "SCHD", "EEMV", "VTV", "JEPI",
    "DIA", "VTI", "VTWO", "QQQ", "NASDX", "SWPPX", "IVV", "SPY", "VOO", "FNILX",
    "HTGC", "JEPQ", "MAIN", "QQQM", "IBIT", "MAGS", "SMH", "USD", "DHT", "FBL",
    "VUG", "VGT", "MGK", "SFY", "XSD", "VEA", "IEFA", "IWF", "IJH", "SBR"
]
    #'Investment': [33.35, 777.60, 296.80, 575.30, 102.72, 48.63, 369.40, 574.00, 1000.00]  # Example values
})

In [8]:
# Calculate average dividend yield for each ETF
etf_features = pd.DataFrame()

for ticker in portfolio['ETF'].to_list():
    price_data = etf_data[ticker]

    # Calculate average dividend yield
    avg_yield = price_data['Dividend_Yield'].mean()

    # Calculate average risk-adjusted score
    avg_risk_adjusted_score = price_data['Risk_Adjusted_Score'].mean()

    # Calculate historical return (percent change over 5 years)
    historical_return = (price_data['Close'].iloc[-1] / price_data['Close'].iloc[0]) - 1

    # Calculate volatility (standard deviation of daily returns)
    volatility = price_data['Close'].pct_change().std()

    # lower expense reatios often provide better net returns
    expense_ratio = stock.info.get('expenseRatio', None)  # Expense ratio of the ETF
    # Growth rate stability
    div_growth_std_dev = price_data['Dividends'].pct_change().std()  # Standard deviation of dividend growth rate


    # find if the dividend yield is stable
    yield_std_dev = price_data['Dividend_Yield'].std()  # Standard deviation of dividend yield

  # Additional metrics
    p_e_ratio = stock.info.get('trailingPE', None)  # Price-to-Earnings Ratio
    p_b_ratio = stock.info.get('priceToBook', None)  # Price-to-Book Ratio
    peg_ratio = stock.info.get('pegRatio', None)  # PEG Ratio
    avg_dividend_growth = price_data['Dividends'].pct_change().mean()  # Dividend Growth Rate
    dividend_payout_ratio = stock.info.get('payoutRatio', None)  # Dividend Payout Ratio
    sharpe_ratio = historical_return / volatility if volatility != 0 else None  # Sharpe Ratio

    # Define the single row of data
    row = pd.DataFrame({
        'ETF': [ticker],  # Use a list for single value
        'Average_Dividend_Yield': [avg_yield],
        'Average_Risk_Adjusted_Score': [avg_risk_adjusted_score],
        'Historical_Return': [historical_return],
        'Volatility': [volatility],
        'P/E_Ratio': [p_e_ratio],
        'P/B_Ratio': [p_b_ratio],
        'PEG_Ratio': [peg_ratio],
        'Dividend_Growth_Rate': [avg_dividend_growth],
        'Dividend_Payout_Ratio': [dividend_payout_ratio],
        'Sharpe_Ratio': [sharpe_ratio]

    })

    # Add the features to the dataframe
    etf_features = pd.concat([etf_features, row])

# Display the feature data


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Define the features and target
X = etf_features[['Average_Dividend_Yield', 'Average_Risk_Adjusted_Score', 'Volatility']]
y = etf_features['Historical_Return']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest regressor
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test data
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest Mean Squared Error: {mse_rf}")

from sklearn.metrics import r2_score
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest R²: {r2_rf}")

from sklearn.metrics import explained_variance_score
explained_variance = explained_variance_score(y_test, y_pred_rf)
print(f"Explained Variance: {explained_variance}")

# Display predicted vs actual returns
predictions_rf = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred_rf
})
#print(predictions_rf)


Random Forest Mean Squared Error: 0.00401368879749526
Random Forest R²: 0.7040874672200477
Explained Variance: 0.7536135878686198


In [10]:
# Predict returns for all ETFs
X_all = etf_features[['Average_Dividend_Yield', 'Average_Risk_Adjusted_Score', 'Volatility']]
etf_features['Predicted_Return'] = rf_model.predict(X_all)

# Assuming etf_features and portfolio are already defined DataFrames
etf_features = pd.merge(etf_features, portfolio, on='ETF', how='inner')

# Calculating Projected_Yield
#etf_features['Projected_Yield'] = etf_features['Average_Dividend_Yield'] * etf_features['Investment']

# Calculate the $ amount: - needs to be more complicated to account for % of the initial investment.
#etf_features['Projected_Earnings'] = etf_features['Predicted_Return'] * initial_investment/len(etf_tickers)

# Rank ETFs by predicted return
best_etfs = etf_features.sort_values(by='Predicted_Return', ascending=False)

#print(best_etfs.info())
# Display top N ETFs
top_n = 10
# Ensure 'best_etfs' DataFrame has 'ETF' and 'Predicted_Return' columns
print(best_etfs[['ETF', 'Predicted_Return','Historical_Return']].head(top_n))
print(best_etfs['ETF'].head(top_n).to_list())


     ETF  Predicted_Return  Historical_Return
29   FBL          0.259038           0.296252
27   USD          0.259038           0.427122
24  IBIT          0.231829           0.221707
25  MAGS          0.152161           0.170531
31   VGT          0.143172           0.143781
32   MGK          0.127407           0.128079
34   XSD          0.125132           0.141635
12  VTWO          0.124713           0.101347
1   RING          0.123759           0.109035
33   SFY          0.123301           0.153315
['FBL', 'USD', 'IBIT', 'MAGS', 'VGT', 'MGK', 'XSD', 'VTWO', 'RING', 'SFY']
